# util-cos

This component provides COS utility functions (e.g. creating a bucket, listing contents of a bucket)

In [2]:
import os

#os.environ['create_image']='True'
os.environ['repository']='docker.io/romeokienzler'
os.environ['version']='0.3'
os.environ['name']='claimed-util-cos'

In [3]:
if bool(os.environ.get('create_image',False)):
    docker_file="""
    FROM debian
    RUN apt update && apt install -y python3-pip procps default-jre
    RUN pip install ipython nbformat aiobotocore botocore s3fs
    ADD util-cos.ipynb .
    CMD ["ipython", "util-cos.ipynb"]
    """
    with open("Dockerfile", "w") as text_file:
        text_file.write(docker_file)

    !docker build -t `echo $name`:`echo $version` .
    !docker tag `echo $name`:`echo $version` `echo $repository`/`echo $name`:`echo $version`
    !docker push `echo $repository`/`echo $name`:`echo $version`
    !rm Dockerfile
else: 
    None

Sending build context to Docker daemon  15.36kB
Step 1/5 : FROM debian
 ---> 54e726b437fb
Step 2/5 : RUN apt update && apt install -y python3-pip procps default-jre
 ---> Using cache
 ---> 378bbf811c49
Step 3/5 : RUN pip install ipython nbformat aiobotocore botocore s3fs
 ---> Using cache
 ---> 579053968a26
Step 4/5 : ADD util-cos.ipynb .
 ---> 7efae8818126
Step 5/5 : CMD ["ipython", "util-cos.ipynb"]
 ---> Running in 190e52ee15a4
Removing intermediate container 190e52ee15a4
 ---> 2695893e143f
Successfully built 2695893e143f
Successfully tagged claimed-util-cos:0.3
The push refers to repository [docker.io/romeokienzler/claimed-util-cos]

f9fb7c28: Preparing 
ea5ff596: Preparing 
05f88123: Preparing 
f9fb7c28: Pushed lready exists 9kBA0.3: digest: sha256:6978b44028ecd7709562182232ab2593e446d9c740d3a21a85aac60f549a2a03 size: 1162


In [ ]:
import logging
import os
import re
import s3fs
import sys

In [ ]:
# access key id
access_key_id = os.environ.get('access_key_id')

# secret access key
secret_access_key = os.environ.get('secret_access_key')

# cos/s3 endpoint
endpoint = os.environ.get('endpoint')

# cos bucket name
bucket_name = os.environ.get('bucket_name')

# path
path = os.environ.get('path','')

# target
target = os.environ.get('target')

# recursive
recursive = bool(os.environ.get('recursive','False'))

# operation
operation = os.environ.get('operation')

In [ ]:
parameters = list(
    map(lambda s: re.sub('$', '"', s),
        map(
            lambda s: s.replace('=', '="'),
            filter(
                lambda s: s.find('=') > -1 and bool(re.match(r'[A-Za-z0-9_]*=[.\/A-Za-z0-9]*', s)),
                sys.argv
            )
    )))

for parameter in parameters:
    logging.debug('Parameter: ' + parameter)
    exec(parameter)

recursive = bool(recursive)

In [ ]:
def print_list(l):
    for file in l:
        print(file)

In [ ]:
s3 = s3fs.S3FileSystem(
    anon=False,
    key=access_key_id,
    secret=secret_access_key,
    client_kwargs={'endpoint_url': endpoint}
)

if operation == 'mkdir':
    s3.mkdir(bucket_name+path)
elif operation == 'ls':
    print_list(s3.ls(bucket_name+path))
elif operation == 'find':
    print_list(s3.find(bucket_name+path))
elif operation == 'get':
    s3.get(bucket_name+path, target, recursive=recursive)
else:
    print(f'operation unkonwn {operation}')